In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import time

class one_news:
    '''Класс для хранения одной новости сайта. Имеет поля для заголовка, времени, тэгов, текста,
    которые все в таком порядке должны входить в конструктор.'''
    def __init__(self, title, date, tags, text):
        self.title = title
        self.date = date
        self.tags = tags
        self.text = text
    
class interview:
    '''Класс для хранения одного интервью. Имеет поля для заголовка, времени, тэгов, имени человека,
    аннотации, вопросов, ответов, которые все в таком порядке должны входить в конструктор.'''
    def __init__(self, title, date, tags, person, annotation, questions, answers):
        self.title = title
        self.date = date
        self.tags = tags
        self.person = person
        self.annotation = annotation
        self.questions = questions
        self.answers = answers
        
class f1news_scraper:
    '''Класс для парсера. Хранит в себе три массива: ссылки на новости, новости, интервью.
    Конструктор не принимает аргументов'''
    def __init__(self):
        self.news_links = []
        self.news = []
        self.interviews = []
        
    def get_all_for_period(self, date_start=dt.datetime.today().date(), date_end=dt.datetime.today().date()):
        '''Метод для сохранения новостей за период времени.
        Аргументами являются два объекта типа datetime: date_start, date_end.
        Период включает свои концы. По умолчанию, скачивает новости за текущий день.
        Данные сохраняются в поля объекта парсера.'''
        curr_date = date_end
        while(curr_date>=date_start):
            self.get_all_for_day(curr_date)
            curr_date = curr_date - dt.timedelta(days=1)
        del curr_date
    
    def get_all_for_day(self, date):
        '''Метод для скачивания новостей за определенный день. 
        В качестве аргумента получает объект типа datetime - date.
        Данные сохраняются в поля объекта парсера.
        '''
        links = self.get_new_links(date)
        for link in links:
            if '/interview' in link:
                try:
                    self.interviews.append(self.get_interview(link))
                except:
                    self.news.append(self.get_one_news(link))
            else:
                try:
                    self.news.append(self.get_one_news(link))
                except:
                    pass
            print('{0} is done'.format(link))
            time.sleep(2)
        del links
    
    def get_new_links(self, date) -> list:
        '''Метод для скачивания ссылок на новости за определенный день. 
        В качестве аргумента получает объект типа datetime - date.
        Возвращает список ссылок.
        '''
        new_links = []
        day_link = 'https://www.f1news.ru/news/{0}/{1}/{2}/'.format(date.year, date.month, date.day)
        page_text = BeautifulSoup(requests.get(day_link).text,"html5lib")
        divs = page_text.findAll('div', {'class':'article_title'})
        links = ["https://www.f1news.ru" + div.findAll('a')[0]['href'] for div in divs]
        for link in links:
            if not ('/archive/' in link) and not (link in self.news_links):
                self.news_links.append(link)
                new_links.append(link)
        del link, day_link, divs, page_text, links
        return new_links        
    
    def get_one_news(self, link) -> one_news:
        '''Скачивает новость по ссылке. Аргументом является link - строка с ссылкой на новость.
        Возвращает объект класса one_news'''
        page_text = BeautifulSoup(requests.get(link).text,"html5lib")
        news = one_news(page_text.find('h1').text, #title
                           dt.datetime.strptime(page_text.find('div', {'class':'post_date'})['content'], '%Y-%m-%dT%H:%M:%S%z'), #date
                           [a.text for a in page_text.find('ul', {'class':'tags'}).findAll('a')], #tags
                           [p.text.strip() for p in page_text.find('div', {'class':'post_content'}).findAll('p')], #text
                           )
        del page_text
        return news
    
    def get_interview(self, link) -> interview:
        '''Скачивает интервью по ссылке. Аргументом является link - строка с ссылкой на новость.
        Возвращает объект класса one_interview'''
        page_text = BeautifulSoup(requests.get(link).text,"html5lib")
        name = page_text.find('h1').text.split(':')[0]
        all_text = ' '.join([p.text.strip() for p in page_text.find('div', {'class':'post_content'}).findAll('p')][1:])
        all_text = all_text.split('Вопрос:')
        questions = []
        answers = []
        for part in all_text:
            if len(part)>0:
                splited = part.split('{0}:'.format(name))
                questions.append(splited[0].strip())
                answers.append(splited[1].strip())
        inter = interview(page_text.find('h1').text, #title
                           dt.datetime.strptime(page_text.find('div', {'class':'post_date'})['content'], '%Y-%m-%dT%H:%M:%S%z'), #date
                           [a.text for a in page_text.find('ul', {'class':'tags'}).findAll('a')], #tags
                           name, #person
                           page_text.find('div', {'class':'post_content'}).find('p').text, #annotation
                           questions, #questions
                           answers) #answers
        del page_text, name, all_text, questions, answers
        return inter
    
    def save_all(self, path):
        '''Сохраняет два файла (один с новостями, второй с интервью).
        Аргумент path - строка с путем до папки с файлами.'''
        self.save_interviews(path)
        self.save_news(path)
        
    def save_news(self, path):
        '''Сохраняет файл с новостями.
        Аргумент path - строка с путем до папки с файлом.'''
        df = pd.DataFrame(columns=['title', 'date', 'tags', 'text'])
        for one_n in self.news:
            df = df.append({'title':one_n.title, 'date':one_n.date, 'tags': one_n.tags, 'text': one_n.text},
                      ignore_index=True )
        df.to_csv(path+'news.csv', sep = ';')
        del df
    
    def save_interviews(self, path):
        '''Сохраняет файл с интервью.
        Аргумент path - строка с путем до папки с файлом.'''
        df = pd.DataFrame(columns=['title', 'date', 'tags', 'person', 'annotation', 'answers', 'questions'])
        for one_i in self.interviews:
            df = df.append({'title':one_i.title, 'date':one_i.date, 'tags': one_i.tags, 'person': one_i.person, 
                       'questions': one_i.questions, 'answers': one_i.answers}, ignore_index=True )
        df.to_csv(path+'interviews.csv', sep = ';')
        del df

scrap = f1news_scraper()
scrap.get_all_for_period(date_start = dt.datetime.today().date() - dt.timedelta(days=7)) #скачиваем новости за неделю
scrap.save_all('') #сохраняем новости в файл

news_df = pd.read_csv('news.csv', sep=';', index_col=0) #читаем и выводим на экран новсти из файлов
print(news_df.head())
inter_df = pd.read_csv('interviews.csv', sep=';', index_col=0)
print(inter_df.head())

https://www.f1news.ru/news/autosport-132464.html is done
https://www.f1news.ru/news/f1-132463.html is done
https://www.f1news.ru/news/f1-132462.html is done
https://www.f1news.ru/news/f1-132461.html is done
https://www.f1news.ru/news/f1-132460.html is done
https://www.f1news.ru/news/f1-132459.html is done
https://www.f1news.ru/news/f1-132458.html is done
https://www.f1news.ru/news/autosport-132465.html is done
https://www.f1news.ru/interview/hartley/132454.shtml is done
https://www.f1news.ru/news/f1-132457.html is done
https://www.f1news.ru/news/f1-132456.html is done
https://www.f1news.ru/news/f1-132455.html is done
https://www.f1news.ru/news/autosport-132453.html is done
https://www.f1news.ru/news/f1-132452.html is done
https://www.f1news.ru/news/f1-132451.html is done
https://www.f1news.ru/news/f1-132450.html is done
https://www.f1news.ru/interview/vandoorne/132449.shtml is done
https://www.f1news.ru/news/f1-132448.html is done
https://www.f1news.ru/news/f1-132445.html is done
https

https://www.f1news.ru/news/f1-132319.html is done
https://www.f1news.ru/interview/gasly/132310.shtml is done
https://www.f1news.ru/interview/alonso/132316.shtml is done
https://www.f1news.ru/interview/bottas/132314.shtml is done
https://www.f1news.ru/Championship/2018/japan/press-thursday.shtml is done
https://www.f1news.ru/interview/max_verstappen/132313.shtml is done
https://www.f1news.ru/interview/vettel/132311.shtml is done
https://www.f1news.ruhttps://www.f1news.ru/gallery/v/2018/gp/japan/0410/ is done
https://www.f1news.ru/news/f1-132309.html is done
https://www.f1news.ru/interview/ricciardo/132308.shtml is done
https://www.f1news.ru/news/f1-132307.html is done
https://www.f1news.ru/interview/grosjean/132305.shtml is done
https://www.f1news.ru/interview/sainz-junior/132306.shtml is done
https://www.f1news.ru/news/f1-132304.html is done
https://www.f1news.ru/news/f1-132303.html is done
https://www.f1news.ru/interview/perez/132302.shtml is done
https://www.f1news.ru/news/f1-132301.